In [1]:
import requests
import pandas as pd
from io import StringIO
from sqlalchemy import create_engine
import psycopg2
import os
import json
from flask import Flask, jsonify



In [2]:
# Database connection parameters
db_params = {
    "dbname": "space_debris_tracker_db",
    "user": "postgres",
    "password": "your_new_password",
    "host": "localhost", 
    "port": "5432"
}

# Establish connection
try:
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    print("Connected to PostgreSQL successfully!")
except Exception as e:
    print("Error connecting to PostgreSQL:", e)

Connected to PostgreSQL successfully!


In [3]:
import requests
import json

# Space-Track login URL and API endpoint
login_url = "https://www.space-track.org/ajaxauth/login"
debris_api = "https://www.space-track.org/basicspacedata/query/class/satcat/limit/100/format/json"

# Space-Track credentials
USERNAME = 'melicontreras32@gmail.com'
PASSWORD = 'MelicarrGrandma!'

# Create a session
session = requests.Session()


# Define headers
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "User-Agent": "Mozilla/5.0"
}

# Define login payload
login_payload = {
    "identity": USERNAME,
    "password": PASSWORD
}

# Perform login request
login_response = session.post(login_url, data=login_payload, headers=headers)

# Debugging: Print login response
print(f"Login Response Code: {login_response.status_code}")
print(f"Login Response Text: {login_response.text}")

# Check if login was successful
if login_response.status_code == 200 and login_response.text.strip():
    print("Login successful.")

    # Fetch data from the API using the session (with stored cookies)
    api_response = session.get(debris_api)

    if api_response.status_code == 200:
        # Parse JSON data
        data = api_response.json()
        print("Data retrieved successfully.")
        
        # Print the first item in the data (for debugging)
        print(json.dumps(data[0], indent=4))
    else:
        print(f"Error fetching data: {api_response.status_code}, {api_response.text}")
else:
    print(f"Login failed: {login_response.status_code}, {login_response.text}")

Login Response Code: 200
Login Response Text: ""
Login successful.
Data retrieved successfully.
{
    "INTLDES": "1957-001A",
    "NORAD_CAT_ID": "1",
    "OBJECT_TYPE": "ROCKET BODY",
    "SATNAME": "SL-1 R/B",
    "COUNTRY": "CIS",
    "LAUNCH": "1957-10-04",
    "SITE": "TTMTR",
    "DECAY": "1957-12-01",
    "PERIOD": "96.19",
    "INCLINATION": "65.10",
    "APOGEE": "938",
    "PERIGEE": "214",
    "COMMENT": null,
    "COMMENTCODE": "4",
    "RCSVALUE": "0",
    "RCS_SIZE": "LARGE",
    "FILE": "1",
    "LAUNCH_YEAR": "1957",
    "LAUNCH_NUM": "1",
    "LAUNCH_PIECE": "A",
    "CURRENT": "Y",
    "OBJECT_NAME": "SL-1 R/B",
    "OBJECT_ID": "1957-001A",
    "OBJECT_NUMBER": "1"
}


In [4]:
# # Space-Track API credentials
# USERNAME = 'melicontreras32@gmail.com'
# PASSWORD = 'MelicarrGrandma!'
# # Space-Track login URL and API endpoint
# login_url = "https://www.space-track.org/ajaxauth/login"
# # debris_api = "https://www.space-track.org/basicspacedata/query/class/gp/format/json"
# debris_api = "https://www.space-track.org/basicspacedata/query/class/satcat/limit/100/format/json"

# # Create a session
# session = requests.Session()

# # Log in to Space-Track and start a session
# login_payload = {'identity': USERNAME, 'password': PASSWORD}
# login_response = session.post(login_url, data=login_payload)

# # Check if login was successful
# if login_response.status_code == 200:
#     print("Login successful.")
#     # Fetch data from the API endpoint
#     api_response = session.get(debris_api)
#     if api_response.status_code == 200:
#         # Parse the JSON data
#         data = api_response.json()
#         print("Data retrieved successfully.")
#         # Example: Print first item in the data
#         print(json.dumps(data[0], indent=4))
#     else:
#         print(f"Error fetching data: {api_response.status_code}, {api_response.text}")
# else:
#     print(f"Login failed: {login_response.status_code}")



In [5]:
query_url = f"{debris_api}/class/satcat/format/csv"

response = session.get(query_url)
    
if response.status_code == 200:
    df = pd.read_csv(StringIO(response.text))
    df.to_csv("satellite_data.csv", index=False)  
    print("Satellite data saved to satellite_data.csv")

else:
    print(f"Error fetching satellite data: {response.status_code}, {response.text}")


Satellite data saved to satellite_data.csv


In [6]:
# Function to fetch debris data
def fetch_debris_data():
    query_url = f"{debris_api}/class/satcat/OBJECT_TYPE/DEBRIS/format/csv"
    response = session.get(query_url)
    
    if response.status_code == 200:
        df = pd.read_csv(StringIO(response.text))
        df.to_csv("debris_data.csv", index=False) 
        print("Debris data saved to debris_data.csv")
        return df
    else:
        print(f"Error fetching debris data: {response.status_code}, {response.text}")
        return pd.DataFrame()
    
    # Fetch space debris data and save it
debris_data = fetch_debris_data()

# Check if data was fetched successfully
if not debris_data.empty:
    print(debris_data.head())  
else:
    print("No debris data received.")

Debris data saved to debris_data.csv
     INTLDES  NORAD_CAT_ID OBJECT_TYPE            SATNAME COUNTRY      LAUNCH  \
0  1960-003C            33      DEBRIS  THOR ABLESTAR DEB      US  1960-04-13   
1  1960-005D            37      DEBRIS      SPUTNIK 4 DEB     CIS  1960-05-15   
2  1960-005E            38      DEBRIS      SPUTNIK 4 DEB     CIS  1960-05-15   
3  1960-005F            39      DEBRIS      SPUTNIK 4 DEB     CIS  1960-05-15   
4  1960-005G            40      DEBRIS      SPUTNIK 4 DEB     CIS  1960-05-15   

    SITE       DECAY  PERIOD  INCLINATION  ...  RCSVALUE  RCS_SIZE FILE  \
0  AFETR  1960-07-17   93.59        51.29  ...         0       NaN    1   
1  TTMTR  1961-06-30   92.92        64.89  ...         0       NaN    1   
2  TTMTR  1960-08-20   90.63        64.89  ...         0       NaN    1   
3  TTMTR  1960-09-24   90.99        64.89  ...         0       NaN    1   
4  TTMTR  1960-09-24   92.13        64.89  ...         0       NaN    1   

   LAUNCH_YEAR  LAUNCH_NU

In [7]:
satellite_data = pd.read_csv("satellite_data.csv")
debris_data = pd.read_csv("debris_data.csv")    

engine=create_engine(f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}/{db_params['dbname']}")

satellite_data.to_sql("satellite_data", engine, if_exists="replace", index=False)
debris_data.to_sql("debris_data", engine, if_exists="replace", index=False)

100

In [8]:
app = Flask(__name__)
@app.route("/get_debris", methods=["GET"])
def get_debris():
    session = requests.Session()

    # Log in to Space-Track
    login_payload = {"identity": USERNAME, "password": PASSWORD}
    login_response = session.post(login_url, data=login_payload)

    if login_response.status_code == 200 and login_response.text.strip():
        print("Login successful.")

        # Fetch space debris data
        response = session.get(data_url)
        if response.status_code == 200:
            debris_data = response.json()
            print("Fetched data:", json.dumps(debris_data, indent=4))  # Log the raw data
            return jsonify(debris_data)  # Return JSON to frontend
        else:
            return jsonify({"error": f"Failed to fetch data: {response.status_code}"}), 500
    else:
        return jsonify({"error": "Login failed"}), 401
